In [12]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt


data1_dir = "../data1.csv"
data2_dir = "../data2.csv"
data3_dir = "../data3.csv"
data4_dir = "../data4.csv"
### 전처리 완료 후 에어포스에 대해서 처리한 데이터 -> processed_data
data_dir='Airfocre_tot.csv'
score_dir='scored_sum_image_normalized.csv'

In [13]:
data1=pd.read_csv(data1_dir,encoding='cp949')
data2=pd.read_csv(data2_dir,encoding='cp949')
data3=pd.read_csv(data3_dir,encoding='cp949')
data4=pd.read_csv(data4_dir,encoding='cp949')

C:\Users\sunwoong\AppData\Local\Temp\ipykernel_3400\3764242080.py:1: DtypeWarning: Columns (22,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv(data1_dir,encoding='cp949')


In [19]:
#중복 id 데이터 전체 제거
df_no_duplicates = data1.drop_duplicates(subset=['id'], keep=False)
df_no_duplicates.head()

,id,status,create_date,name,keyword,price,flag_used,flag_taekpo,flag_exchg,qty,...,uid,is_active,created_at,setl_done_date,purchase_confirm_date,total_price,talk_user_count,first_talk_date,sell_count,register_prod_count
0,3518d5edb4404decebdc3c928f322766,3,2024-02-20 19:42,나이키 문레이서 모나크 295,"나이키문레이서,문레이서모나크,문레이서,295,문레이서295",110000,1,0,0,0,...,NaN,NaN,NaN,NaN,2024-03-25 0:51,113500.0,1.0,2024-03-14,11,1
1,0eb2b66585eefd1d57a21397ca0681e4,3,2024-02-05 12:02,[택포] [280] 나이키 리액티 인피니티 런 플라이니트 런닝화,"나이키리액트,나이키런,나이키런닝화,나이키러닝화,플라이니트",35000,20,1,0,0,...,083a1db0a8f275f9adf14e966a9ceab8,1.0,2023-06-01 6:03,2024-02-13 9:07,2024-02-19 13:54,35000.0,2.0,2024-02-13,1518,1
2,f02a5a22bea44983db7ae19070849068,3,2024-04-19 14:01,에르메스 슬리퍼 이즈미르 42.5,"에르메스,이즈미르,에르메스슬리퍼",400000,20,1,0,0,...,NaN,NaN,NaN,2024-04-22 14:17,2024-04-26 18:09,424000.0,3.0,2024-04-19,3,1
3,30fdc413381ec3bf9a234cfce32e09b9,3,2024-02-15 14:52,에르메스 오란 36.5,"에르메스,오란,에르메스오란,오란블랙,오란검정",500000,21,0,0,0,...,NaN,NaN,NaN,NaN,2024-03-26 15:51,450000.0,4.0,2024-03-01,9,1
4,bd17645bfa0ff30741d56cf9efbfc609,3,2024-01-26 12:40,아디다스 리프시몬스 275 박스풀 미개봉,라프시몬스,130000,2,1,0,1,...,93c97e8b3fc1511f3a47db1b12542b49,0.0,2022-12-19 20:20,2024-01-29 23:47,2024-02-01 19:16,13000.0,1.0,2024-01-28,31,1


In [20]:
#판매 기간을 위한 데이터 변환
df_no_duplicates['setl_done_date'] = pd.to_datetime(df_no_duplicates['setl_done_date'], errors='coerce')
df_no_duplicates['purchase_confirm_date'] = pd.to_datetime(df_no_duplicates['purchase_confirm_date'], errors='coerce')
df_no_duplicates['final_purchase_date'] = df_no_duplicates[['setl_done_date', 'purchase_confirm_date']].min(axis=1)
#판매 기간 계산
df_no_duplicates['create_date'] = pd.to_datetime(df_no_duplicates['create_date'], errors='coerce')
df_no_duplicates['first_talk_date'] = pd.to_datetime(df_no_duplicates['first_talk_date'], errors='coerce')
df_no_duplicates['purchase_duration'] = (df_no_duplicates['final_purchase_date'] - df_no_duplicates['create_date']).dt.total_seconds() / 3600
#df_cleaned['purchase_duration'] = (df_cleaned['final_purchase_date'] - df_cleaned['create_date']).dt.days
df_no_duplicates['talk_duration'] = (df_no_duplicates['first_talk_date'] - df_no_duplicates['create_date']).dt.days


C:\Users\sunwoong\AppData\Local\Temp\ipykernel_3400\1554480675.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['setl_done_date'] = pd.to_datetime(df_no_duplicates['setl_done_date'], errors='coerce')
C:\Users\sunwoong\AppData\Local\Temp\ipykernel_3400\1554480675.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['purchase_confirm_date'] = pd.to_datetime(df_no_duplicates['purchase_confirm_date'], errors='coerce')
C:\Users\sunwoong\AppData\Local\Temp\ipykernel_3400\155448

In [40]:
product_list = ['아디다스 삼바 OG 클라우드 화이트', '반스 올드스쿨 블랙', '뉴발란스 993 메이드 인 USA 그레이', '컨버스 척 70 하이 블랙', '컨버스 척 70 로우 블랙', '조던 1 x 트래비스 스캇 x 프라그먼트 레트로 로우 OG SP 밀리터리 블루', "나이키 에어포스 1 '07 WB 플랙스", '팀버랜드 프리미엄 6인치 워터프루프 부츠 위트 누벅', '미하라 야스히로 피터슨 OG 솔 캔버스 로우 스니커즈 블랙', '아디다스 슈퍼스타 코어 블랙 화이트', "나이키 에어포스 1 '07 로우 트리플 블랙", '뉴발란스 992 메이드 인 USA 그레이', "나이키 에어포스 1 '07 로우 화이트", '나이키 덩크 로우 레트로 블랙', '아디다스 삼바 OG 블랙 화이트 검']

In [43]:

# name_kor 컬럼이 product_list에 있는 데이터만 필터링
df_filtered_products = df_no_duplicates[df_no_duplicates['name_kor'].isin(product_list)]
df_filtered_products.head()

,id,status,create_date,name,keyword,price,flag_used,flag_taekpo,flag_exchg,qty,...,setl_done_date,purchase_confirm_date,total_price,talk_user_count,first_talk_date,sell_count,register_prod_count,final_purchase_date,purchase_duration,talk_duration
5,cbe6ea5cd834142383f0c2e7ca19e6b8,3,2024-04-29 21:09:00,나이키 범고래 덩크 240,"나이키,조던,범고래,240",40000,21,1,0,0,...,NaT,NaT,40000.0,2.0,2024-05-27,8,1,NaT,NaN,27.0
20,96e84a70cd5325fdeb4b84498331ad5b,3,2024-04-13 11:28:00,"뉴발란스 992 그레이[280] 상태 매우좋습니다(사진첨부,번개케어가능)","뉴발란스,뉴발란스992,992,992그레이,992280",339000,1,1,0,0,...,2024-04-13 18:12:00,2024-04-23 17:07:00,360860.0,3.0,2024-04-13,1,1,2024-04-13 18:12:00,6.733333,-1.0
74,1795a0b5fcae0670f7cdefa4903fe170,3,2024-02-27 09:40:00,아디다스 삼바 OG 블랙 235,"아디다스,아디다스삼바,아디다스삼바235,아디다스가젤,아디다스스페지알",100000,20,0,0,0,...,NaT,2024-03-04 14:51:00,100000.0,1.0,2024-02-27,18,1,2024-03-04 14:51:00,149.183333,-1.0
146,1e6f126212277d7f5db664c54dca3b0c,3,2024-01-13 20:23:00,컨버스 척테일러 162050c,"컨버스,척테일러",49000,2,1,0,0,...,2024-01-17 11:40:00,2024-01-19 19:35:00,50710.0,1.0,2024-01-17,2,1,2024-01-17 11:40:00,87.283333,3.0
189,9d102d7cb403e706afd72802c569c4f6,3,2024-03-05 20:59:00,[260] 뉴발란스 993 그레이,"뉴발란스,993,260",300000,20,1,0,0,...,NaT,2024-03-13 15:51:00,300000.0,2.0,2024-03-06,1,1,2024-03-13 15:51:00,186.866667,0.0
